In [1]:
import psychopy.filters
from tkinter import *
from tkinter import ttk
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from PIL import ImageDraw
import os

pygame 2.0.1 (SDL 2.0.14, Python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
max_val = 256
new_dir_path = 'pre_spec'+str(max_val)
try:
    os.makedirs(new_dir_path)
except FileExistsError:
    print('フォルダーが既に存在したためスルーしました')

In [11]:
alp =["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q","R","S","T","U","V","W","X","Y","Z"]
for i in range(len(alp)):
    original_image_path = '../修論実験/Images/'+alp[i]+'256.png'

    phase_shifted_f_uv = img_shiftedFuv(original_image_path)

    new_dir_path_folder = new_dir_path + "/" +alp[i]
    try:
        os.makedirs(new_dir_path_folder)
    except FileExistsError:
        print('フォルダーが既に存在したためスルーしました')

    for radius in range(3,8):
        filter_save(phase_shifted_f_uv, radius, new_dir_path_folder)

フォルダーが既に存在したためスルーしました


In [9]:
def img_shiftedFuv(img_file):
    img = Image.open(img_file)
    gray_img = img.convert('L')
    f_xy = np.asarray(gray_img)
    f_uv = np.fft.fft2(f_xy)
    shifted_f_uv = np.fft.fftshift(f_uv)
    return shifted_f_uv

In [10]:
def make_filer(ellipse_percentage, shifted_f_uv):
    # フィルタ (ローパス) を用意する
    x_pass_filter = Image.new(mode='L',  # 8-bit pixels, black and white
                              size=(shifted_f_uv.shape[0],
                                    shifted_f_uv.shape[1]),
                              color=0,  # default black
                              )
    # 中心に円を描く
    draw = ImageDraw.Draw(x_pass_filter)
    # 円の半径
    ellipse_r = shifted_f_uv.shape[0] * ellipse_percentage
    # 画像の中心
    center = (shifted_f_uv.shape[0] // 2,
              shifted_f_uv.shape[1] // 2)
    # 円の座標
    ellipse_pos = (center[0] - ellipse_r,
                   center[1] - ellipse_r,
                   center[0] + ellipse_r,
                   center[1] + ellipse_r)
    draw.ellipse(ellipse_pos, fill=255)
    # フィルタ
    filter_array = np.asarray(x_pass_filter)
    return filter_array

In [6]:
def filter_save(shifted_f_uv, val, save_path):
    ###
    if val == 0:
        val =1
    lp_filt = psychopy.filters.butter2d_lp(
        size=(256,256),
        cutoff=val/512,
        n=4)
    filtered_f_uv = np.multiply(shifted_f_uv, lp_filt)
    unshifted_f_uv = np.fft.fftshift(filtered_f_uv)
    i_f_xy = np.fft.ifft2(unshifted_f_uv).real
    ###
    
    ###
    filter_not_butter = make_filer(val/512, shifted_f_uv)
    filtered_f_uv_not_butter = np.multiply(shifted_f_uv, filter_not_butter)
    unshifted_f_uv_not_butter = np.fft.fftshift(filtered_f_uv_not_butter)
    i_f_xy_not_butter = np.fft.ifft2(unshifted_f_uv_not_butter).real
    ###
    
#     fig, axes = plt.subplots(2, 2, figsize=(12, 6))
    
#     axes[0, 0].imshow(lp_filt, cmap='gray')
#     axes[0, 0].set_title('Butterworth Filter')
#     axes[0, 1].imshow(i_f_xy, cmap='gray')
#     axes[0, 1].set_title('Butterworth Image')
    
#     axes[1, 0].imshow(filter_not_butter, cmap='gray')
#     axes[1, 0].set_title('spectrum0_1 Filter')
#     axes[1, 1].imshow(i_f_xy_not_butter, cmap='gray')
#     axes[1, 1].set_title('spectrum0_1 Image')
#     plt.show()
#     plt.savefig(new_dir_path+'/'+str(val)+'.png')

#     plt.imsave(save_path+'/'+str(val)+'_0.png',lp_filt)
    plt.imsave(save_path+'/'+str(val)+'_1.png',i_f_xy)
#     plt.imsave(save_path+'/'+str(val)+'_2.png',filter_not_butter)
    plt.imsave(save_path+'/'+str(val)+'_3.png',i_f_xy_not_butter)